Autores:
- Adniel Quintana Muñoz
- Carlos Cesar Caballero Díaz
- Marisleydis Alvarez Noy

# Introducción

Informalmente el valor de la integral de una función $f(x)$ en el intervalo [a, b] es igual al área de la región limitada por la curva de $f(x)$, el eje $x$, y las rectas verticales $x = a$ y $x = b$. Esta área puede aproximarse mediante la suma de las áreas de los $n$ rectángulos:

\begin{align}
\int_a^b f(x)dx \cong h \sum_{j=0}^{n-1} f(x_j)
\end{align}

donde $h = (b − a)/n$ es el paso de integración, $n$ es el número de sub-intervalos de integración, $yx_j = a + jh$ para $j = 0,1, ... , n − 1$ son los puntos dentro del intervalo de integración.

Para el desarrollo del presente trabajo se implementaron en el lenguaje de programación Python en su versión 3.7.5, un algoritmo secuencial, y dos algoritmos paralelos utilizando multiprocessing y mpi4py, con el objetivo de analizar los resultados. Los experimentos se ejecutaron en un equipo con un procesador Intel® Core™ i7-8550U CPU @ 1.80GHz × 8 y 15,4 GiB de memoria ram disponibles sobre el sistema operativo Ubuntu en su versión 19.10. Los algoritmos paralelos se ejecutaron utilizando 4 núcleos de procesamiento.

# Desarrollo

Para el desarrollo del trabajo se realizaron tres implementaciones, una secuencial, una paralela en memoria utilizando el modulo multiprocessing (equivalente a OpenMP para Python) de la biblioteca estándar y otra implementación paralela utilizando el módulo mpi4py, el cual implementa MPI para este lenguaje de programación.

## Integración secuencial

Para la ejecución secuencial se implementó el algoritmo de integración numérica descrito en el ejercicio de la siguiente forma:

In [10]:
def secuential_integrate(a,b,n,func):
    #se calcula el ancho
    h = (b-a)/n
    i = 0.0
    # se evalúa la función para cada uno de los fragmentos y se suman los resultados
    for j in range(n):
        i = i + func(a+j*h)
    # los resultados de la ecaluación de la función se multiplican por h para obtener el valor de la integral
    i = i*h
    return i

El algoritmo calcula el valor de la integral para la función `func` en el intervalo comprendido entre `a` y `b` para `n` rectángulos

## Integración paralela

Para la implementación paralela del algoritmo de integración en primer lugar se realizó una división del problema en partes de forma tal que se pudieran distribuir a múltiples tareas para que puedan trabajar en el problema simultáneamente mediante una descomposición de dominio en forma de bloques de una dimensión.

Se elije realizar una descomposición de dominio porque es posible dividir el problema en partes iguales que realicen las mismas tareas (o pedazos de una misma tarea) en distintos núcleos de procesamiento.



![title](assets/integral-parallel.png)

### Multiprocessing (equivalente a OpenMP)

Para la ejecución paralela con multiprocessing (equivalente a OpenMP para el lenguaje de programación Python) se implementó el algoritmo de integración numérica descrito en el ejercicio de la siguiente forma:

In [11]:
_func = None

def multiprocessing_integral(a, h, n, rank):
    integ = 0.0
    for j in range(n):
        a_j = a + (j+n*rank) * h
        integ += _func(a_j)
    return integ


def multiprocessing_integrate(a, b, n, func):
    # la asignación de _func permitirá usar la lambda func con multiprocessing
    global _func
    _func = func

    # size define la cantidad de procesos a ejecutar
    size = 4
    
    # se divide n segun size
    n = n//size
    #se calcula el ancho
    h = (b - a) / (n * size)

    integral_values = []
    # se declara un Pool para 'size' procesos
    with Pool(size) as p:
        # se instancia una funcion parcial para permitir multiples argumentos a multiprocessing 
        func = partial(multiprocessing_integral, a, h, n)
        # se ejecutan los procesos y se espera por los resultados
        integral_values = p.map(func, range(size))
    integral_sum = 0.0
    #se suman los resultados y se multiplica por h
    for integral_value in integral_values:
        integral_sum += integral_value
    integral_sum = integral_sum * h
    return integral_sum

### MPI (mpi4py)

Para la ejecución paralela con MPI se implementó el algoritmo de integración numérica descrito en el ejercicio de la siguiente forma:

In [12]:
def mpi_integrate(a, b, n, func):
    # se obtiene el intracommunicator COMM_WORLD
    comm = MPI.COMM_WORLD
    # se obtiene el proceso actual
    rank = comm.Get_rank()
    # se obtiene el total de procesos
    size = comm.Get_size()

    # se divide n segun size
    n = n//size

    def integral(a, h, n):
        integ = 0.0
        for j in range(n):
            a_j = a + (j+n*rank) * h
            integ += func(a_j)
        return integ

    #se calcula el ancho
    h = (b - a) / (n * size)

    # se calcula la integral para el proceso actual
    my_int = numpy.full(1, integral(a, h, n))

    
    if rank == 0:
        # si es el proceso 0, se dispone a obtener los datos de los otros procesos 
        integral_sum = my_int[0]
        # ciclo en el que se obtienen los datos de los otros procesos y se termina el calculo de la integral
        for p in range(1, size):
            comm.Recv(my_int, source=p)
            integral_sum += my_int[0]
        integral_sum = integral_sum * h

        print(integral_sum)
    else:
        # si no es el proceso 0, evía el resultado al proceso 0
        comm.Send(my_int, dest=0)

## Ejecución

Para la ejecución del experimento se conformó un script en bash el cual realiza diez (10) ejecuciones para cada uno de los casos de pruebas definidos, en cuya salida se pueden obtener los siguientes valores:

- mem: Porciento de memoria del sistema utilizada por la ejecución
- RSS: Máxima memoria utilizada durante la ejecución
- time: tiempo de ejecución
- cpu.sys: Número total de segundos-CPU utilizados

Para las funciones:

- 1: $f(x) = x^2+cos(x)$
- 2: $f(x) = (8*e^{(1-x)})+(7*log(x))$
- 3: $f(x) = 10+x^2-10*cos^2(\pi*x)^2$

Arrojando el siguiente resultado:

    Función 1, n=10000

    ---- secuencial -----
    mem=0 RSS=37796 time=0:06.16 cpu.sys=5.13

    ---- mpi -----
    mem=0 RSS=38172 time=0:07.44 cpu.sys=8.33

    ---- multiprocessing -----
    mem=0 RSS=38672 time=0:07.34 cpu.sys=5.21
    -------------------------------

    Función 1, n=100000

    ---- secuencial -----
    mem=0 RSS=37684 time=0:06.61 cpu.sys=4.96

    ---- mpi -----
    mem=0 RSS=38296 time=0:07.49 cpu.sys=8.31

    ---- multiprocessing -----
    mem=0 RSS=38632 time=0:07.33 cpu.sys=5.39
    -------------------------------

    Función 1, n=1000000

    ---- secuencial -----
    mem=0 RSS=37740 time=0:10.62 cpu.sys=4.96

    ---- mpi -----
    mem=0 RSS=38276 time=0:08.96 cpu.sys=8.00

    ---- multiprocessing -----
    mem=0 RSS=38680 time=0:08.41 cpu.sys=5.44
    -------------------------------

    Función 2, n=10000

    ---- secuencial -----
    mem=0 RSS=37708 time=0:06.13 cpu.sys=4.95

    ---- mpi -----
    mem=0 RSS=38184 time=0:07.49 cpu.sys=8.23

    ---- multiprocessing -----
    mem=0 RSS=38796 time=0:07.34 cpu.sys=5.27
    -------------------------------

    Función 2, n=100000

    ---- secuencial -----
    mem=0 RSS=37768 time=0:06.80 cpu.sys=5.03

    ---- mpi -----
    mem=0 RSS=38124 time=0:07.80 cpu.sys=8.50

    ---- multiprocessing -----
    mem=0 RSS=38748 time=0:07.30 cpu.sys=5.36
    -------------------------------

    Función 2, n=1000000

    ---- secuencial -----
    mem=0 RSS=37696 time=0:13.07 cpu.sys=4.96

    ---- mpi -----
    mem=0 RSS=38196 time=0:09.55 cpu.sys=8.29

    ---- multiprocessing -----
    mem=0 RSS=38760 time=0:09.30 cpu.sys=5.24
    -------------------------------

    Función 3, n=10000

    ---- secuencial -----
    mem=0 RSS=37764 time=0:06.17 cpu.sys=5.18

    ---- mpi -----
    mem=0 RSS=38204 time=0:07.62 cpu.sys=8.23

    ---- multiprocessing -----
    mem=0 RSS=38828 time=0:07.34 cpu.sys=5.24
    -------------------------------

    Función 3, n=100000

    ---- secuencial -----
    mem=0 RSS=37740 time=0:06.80 cpu.sys=4.87

    ---- mpi -----
    mem=0 RSS=38308 time=0:07.78 cpu.sys=8.58

    ---- multiprocessing -----
    mem=0 RSS=38576 time=0:07.36 cpu.sys=5.17
    -------------------------------

    Función 3, n=1000000

    ---- secuencial -----
    mem=0 RSS=37816 time=0:13.23 cpu.sys=5.09

    ---- mpi -----
    mem=0 RSS=38292 time=0:09.65 cpu.sys=8.43

    ---- multiprocessing -----
    mem=0 RSS=38756 time=0:09.61 cpu.sys=5.43
    -------------------------------

# Conclusiones

Tras los resultados experimentales se puede concluir que, para resultados con `n=10000` el algoritmo secuencial presenta menores tiempos de ejecución, seguido de la implementación con multiprocesing y luego MPI. Para los resultados con `n=100000` el algoritmo secuencial continúa presentando menores tiempos de ejecución aunque con una menor diferencia con respecto a la implementación utilizando multiprocessing y continuando la implementación MPI con los mayores tiempos de ejecución. Para los resultados con `n=1000000` se comienzan a ver las ventajas de la implementación paralela, teniendo multiprocessing los menores tiempos de ejecución, seguido por MPI, ambos presentando diferencias notables con respecto a la implementación secuencial.

El comportamiento en cuanto al consumo de memoria se aprecia de forma similar en las distintas ejecuciones, presentando la implementación secuencial un menor consumo de memoria, seguido de la implementación MPI.

# Anexos

- implementación: https://github.com/cccaballero/miav-programacion-avanzada